In [1]:
import os
!pwd
os.chdir('../')

/Users/shrey/Desktop/github/butterfly_classifier/research


In [2]:
!pwd

/Users/shrey/Desktop/github/butterfly_classifier


In [3]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

model = tf.keras.models.load_model('artifacts/training/model.h5')


from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data_dir: Path
    training_data_csv: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [4]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_dir, save_json


class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_dir([self.config.artifacts_root])


    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model='artifacts/training/model.h5',
            training_data_dir='artifacts/data_ingestion/data/train',
            training_data_csv='artifacts/data_ingestion/data/Training_set.csv',
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [5]:
from urllib.parse import urlparse
import pandas as pd


class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        valid_df = pd.read_csv(self.config.training_data_csv)

        valid_df['filename'] = valid_df['filename'].apply(
            lambda x: os.path.join(self.config.training_data_dir, x))
        


        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.3
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            class_mode='categorical',
            interpolation='bilinear',
            shuffle=False
        )

        valid_datagenrator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs)
        
        self._valid_generator = valid_datagenrator.flow_from_dataframe(
            dataframe=valid_df,
            x_col='filename',
            y_col='label',
            subset='validation',
            **dataflow_kwargs
        )
        

        
        # self._valid_generator = valid_datagenrator.flow_from_directory(
        #     directory=self.config.training_data,
        #     subset='validation',
        #     shuffle=False,
        #     **dataflow_kwargs
        # )


    @staticmethod
    def load_model(path: Path) -> tf.keras.models.Model:
        return tf.keras.models.load_model(path)
        
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self._valid_generator)


    def save_score(self):
        scores = {'loss': self.score[0], 'accuracy': self.score[1]}
        save_json(path=Path('scores.json'), data=scores)

        

In [6]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
    raise e 

Found 1949 validated image filenames belonging to 75 classes.


/opt/anaconda3/envs/flyenv/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


122/122 ━━━━━━━━━━━━━━━━━━━━ 149s 1s/step - accuracy: 0.5957 - loss: 1.9226
